In [3]:
import pandas as pd
import geopandas as gpd

In [6]:
gdf = gpd.read_file('data/geometry/world-administrative-boundaries.geojson')

In [61]:
gdf.columns

Index(['geo_point_2d', 'iso3', 'status', 'color_code', 'name', 'continent',
       'region', 'iso_3166_1_alpha_2_codes', 'french_short', 'geometry'],
      dtype='object')

In [86]:
gdfc = gdf[['name','region','continent','geometry']].sort_values('name').set_index('name')

In [19]:
df = pd.read_csv('data/iucn/iucn_init.csv')

In [21]:
df.columns

Index(['sci_name', 'legend', 'source', 'event_year', 'longitude', 'latitude'], dtype='object')

In [22]:
iucn = gpd.GeoDataFrame(df[['sci_name','legend']],geometry=gpd.points_from_xy(df.longitude,df.latitude), crs=4326)

In [23]:
iucn

,sci_name,legend,geometry
0,Hubbsina turneri,Extant (resident),POINT (-101.47950 19.87450)
1,Hubbsina turneri,Extant (resident),POINT (-101.77730 19.82560)
2,Hubbsina turneri,Extant (resident),POINT (-101.78760 19.82730)
3,Ictalurus mexicanus,Extant (resident),POINT (-99.35417 21.98083)
4,Ictalurus mexicanus,Extant (resident),POINT (-99.30000 22.00000)
...,...,...,...
46429,Holoscolex dossantosi,Extant (resident),POINT (-46.75333 -3.69000)
46430,Pontoscolex pydanieli,Extant (resident),POINT (-64.43333 -10.30000)
46431,Pontoscolex pydanieli,Extant (resident),POINT (-64.43333 -10.30000)
46432,Pontoscolex pydanieli,Extant (resident),POINT (-64.40194 -9.26083)


In [74]:
status = [*iucn.legend.unique()]
status

['Extant (resident)',
 'Possibly Extinct',
 'Presence Uncertain',
 'Possibly Extant (resident)',
 'Extant & Vagrant (resident)',
 'Possibly Extinct & Vagrant',
 'Extinct']

In [87]:
def asSet(list):
    return set(list);

In [149]:
names = []
giucn = gdfc.copy()
for s in iucn.legend.unique():
    d1 = gpd.sjoin(iucn.drop(columns=['legend']).loc[iucn.legend==s],gdfc[['geometry']], how='left', predicate='within').dropna()
    d2 = d1.dissolve(by='index_right', aggfunc=asSet)
    d3 = pd.DataFrame({'name':[*d2.index.values],s:[list(s) for s in d2.sci_name.values],
                       f'point data {status.index(s)+1}':[*d2.geometry.to_wkt()]}).set_index('name')
    giucn = giucn.join(d3)
    names.extend([*d3.index])

In [150]:
giucn = giucn.loc[giucn.index.isin(names)]

In [151]:
giucn

,region,continent,geometry,Extant (resident),point data 1,Possibly Extinct,point data 2,Presence Uncertain,point data 3,Possibly Extant (resident),point data 4,Extant & Vagrant (resident),point data 5,Possibly Extinct & Vagrant,point data 6,Extinct,point data 7
name,,,,,,,,,,,,,,,,,
Albania,Southern Europe,Europe,"POLYGON ((20.07142 42.56091, 20.10208 42.53347...",[Luciola novaki],"MULTIPOINT (19.3539 41.9426, 19.398 41.867, 19...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,Northern Africa,Africa,"POLYGON ((8.62203 36.94137, 8.63222 36.88194, ...",[Calopteryx exul],"MULTIPOINT (7.30867 36.4419, 7.31001 36.2411, ...",NaN,NaN,[Calopteryx exul],"MULTIPOINT (-1.34306 34.6359, 2.76438 36.3584,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,Middle Africa,Africa,"MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -...",[Agriocnemis bumhilli],"MULTIPOINT (17.8148 -14.4393, 18.1706 -14.5899...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,South America,Americas,"MULTIPOLYGON (((-68.64312 -54.88861, -68.63723...","[Nantis indefessus, Australoheros ykeregua, Cn...","MULTIPOINT (-66.2839 -25.4394, -66.1583 -25.12...",NaN,NaN,[Scleronema operculatum],POINT (-57.81 -28.85),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Australia,Australia and New Zealand,Oceania,"MULTIPOLYGON (((158.88217 -54.71139, 158.87966...","[Nunciella kangarooensis, Synemon ignita, Chor...","MULTIPOINT (130.65 -11.4333, 130.656 -11.3792,...","[Euryglossina globuliceps, Ogyris halmaturia, ...","MULTIPOINT (133.427 -31.903, 141.601 -38.34, 1...",[Keyacris scurra],"MULTIPOINT (145.32 -36.7694, 145.611 -37.0244,...",NaN,NaN,NaN,NaN,NaN,NaN,"[Xylocopa aerata, Ogyris halmaturia, Keyacris ...","MULTIPOINT (138.612 -35.02, 140.466 -36.167, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vanuatu,Melanesia,Oceania,"MULTIPOLYGON (((169.84607 -20.25223, 169.82690...",[Birgus latro],"MULTIPOINT (167.153 -15.1979, 168.31 -17.7278)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Venezuela,South America,Americas,"MULTIPOLYGON (((-61.18507 8.49674, -61.19417 8...","[Laimosemion lyricauda, Hyphessobrycon fernand...","MULTIPOINT (-72.6294 8.60672, -72.6294 8.60672...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Moenkhausia pittieri],"MULTIPOINT (-67.9025 10.1581, -67.7431 10.1796..."
Vietnam,South-Eastern Asia,Asia,"MULTIPOLYGON (((104.08288 10.36486, 104.08663 ...","[Ophiogomphus minimus, Coeliccia natgeo, Coeli...","MULTIPOINT (103.747 22.3476, 103.936 10.354, 1...",[Protosticta spinosa],POINT (104.34 21.733),[Drepanosticta emtrai],POINT (105.48 20.784),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
endangereds=[]
for n in giucn.index:
    endangered=[]
    for s in status:
        if isinstance(giucn.loc[n,s], list):
            endangered.append({'status':s,'species':giucn.loc[n,s],
                               'locations':giucn.loc[n,f'point data {status.index(s)+1}']})
    endangereds.append(str(endangered))
giucn['endangered'] = endangereds

In [160]:
giucnc = giucn[['region','continent','endangered','geometry']]
giucnc

,region,continent,endangered,geometry
name,,,,
Albania,Southern Europe,Europe,"[{'status': 'Extant (resident)', 'species': ['...","POLYGON ((20.07142 42.56091, 20.10208 42.53347..."
Algeria,Northern Africa,Africa,"[{'status': 'Extant (resident)', 'species': ['...","POLYGON ((8.62203 36.94137, 8.63222 36.88194, ..."
Angola,Middle Africa,Africa,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -..."
Argentina,South America,Americas,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((-68.64312 -54.88861, -68.63723..."
Australia,Australia and New Zealand,Oceania,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((158.88217 -54.71139, 158.87966..."
...,...,...,...,...
Vanuatu,Melanesia,Oceania,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((169.84607 -20.25223, 169.82690..."
Venezuela,South America,Americas,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((-61.18507 8.49674, -61.19417 8..."
Vietnam,South-Eastern Asia,Asia,"[{'status': 'Extant (resident)', 'species': ['...","MULTIPOLYGON (((104.08288 10.36486, 104.08663 ..."


In [161]:
giucnc.to_file('data/geojson/iucn.geojson')